<a href="https://colab.research.google.com/github/luis-vizcaya-vr/Imports/blob/Features-LR/dart_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install darts
!pip install pyyaml==5.4.1
!pip install matplotlib==3.1.3

In [ ]:
import pandas as pd
from darts import TimeSeries
import matplotlib.pyplot as plt
import darts as drt
FORECAST_PERIOD = 14

df = pd.read_csv('natgas.csv', usecols= [0,1])
df['Date1'] = pd.date_range(start='9/1/2021', end = '02/22/2022', freq='D')

series = TimeSeries.from_dataframe(df, time_col = 'Date1',  value_cols ='NatGas - Transco Z6 Non-NY', freq = 'd', fill_missing_dates = True)
train, val = series[:-FORECAST_PERIOD], series[-FORECAST_PERIOD:]

def eval_model(model,train):
  res = {}
  print('Running: ', model)
  model.fit(train)
  forecast = model.predict(len(val))
  #print("model {} obtains MAE: {:.2f}".format(model, drt.metrics.metrics.mae(val, forecast)))
  res['MAPE'] = drt.metrics.metrics.mape(val, forecast)
  res['Forecast'] = forecast
  return res

def Best_model(model_list, train,metric = 'MAPE'):
  Best = ''
  Plot_Id = 100
  results = {}
  Max_Dev = float("Inf")
  for m in model_list:
    r = eval_model(m,train)
    Plot_Id += 1
    print(metric,' :', r[metric])
    if r[metric] < Max_Dev:
      Max_Dev = r[metric]
      results = r
      results['Model'] = str(m)
  return results




In [ ]:
from darts.models import *

model_list = [ExponentialSmoothing(),FFT(),ARIMA(),AutoARIMA(),KalmanForecaster()]
a = Best_model(model_list, train)
print('Best Model: ', a['Model'])
a['Forecast'].plot()
val.plot()
train.plot()


'''
model.fit(train)
prediction = model.predict(len(val), num_samples=1)
series.plot()
#prediction.plot(label='forecast', low_quantile=0.1, high_quantile=0.9)
prediction.plot(label='forecast')
plt.legend()
print(drt.metrics.metrics.mae(val, prediction))
'''